In [1]:
import pandas as pd
import statsmodels.api as sm
import datetime
import sklearn as sk
import numpy as np
import seaborn as sn

# Load Wimmer dataset to use as base df

In [2]:
wim = pd.read_csv('./sources/WimmerMin.csv')
# Remove all non-coldwar entries
mask = wim.year > 1945 
wim = wim[mask]
mask = wim.year < 1992
wim = wim[mask]
# Wimmers war data, when ignored, will cause duplicate rows to appear, remove them
wim.drop_duplicates(subset=['year', 'cowcode'], keep='first', inplace=True)

### Add US political information (Dummies for presidents and parties)

In [3]:
# Convert start to datetime
pres = pd.read_csv('./sources/us_presidents.csv')
pres['start'] = pd.to_datetime(pres['start'])
# Filter out non relevant presidents

# Convert date to year
pres['start'] = pres['start'].dt.year
# Remove non-CW dates
mask = pres['start'] >= 1945
pres = pres[mask]
mask = pres['start'] < 1992
pres = pres[mask]

# Set Truman start to 1946 for data reasons
pres = pres[['start', 'president', 'party']]
pres[:1].start = pres[:1].start +1
dummy_pres = pres['president'].str.get_dummies()
dummy_party = pres['party'].str.get_dummies()
pres = pd.concat([pres, dummy_pres], axis=1)
pres = pd.concat([pres, dummy_party], axis=1)
pres = pres[['start', 'president', 'party', 'Dwight D. Eisenhower', 'George H. W. Bush', 
'Gerald Ford', 'Harry S. Truman', 'Jimmy Carter',
'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon',
'Ronald Reagan', 'Democratic', 'Republican']]
    
wim = wim.merge(pres, left_on='year', right_on='start', how='left')
wim[[
    'president', 'party',
    'Dwight D. Eisenhower', 'George H. W. Bush', 
    'Gerald Ford', 'Harry S. Truman', 'Jimmy Carter',
    'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon',
    'Ronald Reagan', 'Democratic', 'Republican']] = wim[
    [
        'president', 'party',
        'Dwight D. Eisenhower', 'George H. W. Bush', 
        'Gerald Ford', 'Harry S. Truman', 'Jimmy Carter',
        'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon',
        'Ronald Reagan', 'Democratic', 'Republican'
    ]].ffill()


# Populate A variable that tracks the current US leaders years in office
wim['start'] = wim['start'].ffill()
wim['USP_YIO'] = wim['year'] - wim['start'] 

/home/jasper/Dropbox/ULTRADOCUMENTEN/THESIS_CIVIL_WAR/THESIS/venv/lib/python3.5/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Add Soviet Political Information

In [4]:
# Malenkov added, but should be excluded from most analysis
# The dict is WRONG ofcourse (Lenin missing, Stalin his start date is incorrect)
# But this data is only meant to merge with the wimmer database, which starts at 1946
# Malenkov is removed, not just because of his short tenure 
#(which would apply to multiple leaders)
# But because his was striped of most of his power before he was deposed

su_leader = {
    'Stalin': 1946, 'Khrushchev': 1953, 'Brezhnev': 1964, 'Andropov': 1982,
    'Chernenko': 1984, 'Gorbachev': 1985
}

# Convert dict to dataframe
su_leaders = pd.DataFrame.from_dict(
    su_leader, orient='index').reset_index()
# Rename columns and make sure df is properly sorted
su_leaders.columns = ['SU_leader', 'SUL_start_year']
su_leaders.sort_values('SUL_start_year', inplace=True)
# Generate dummies and merge them in the su_leaders dataframe
dummies = su_leaders['SU_leader'].str.get_dummies()
su_leaders= pd.concat([su_leaders, dummies], axis=1)

# Merge df with wimmers main dataset. 
wim = wim.merge(su_leaders, left_on='year', right_on='SUL_start_year', how='left')
#ffill empty values
wim[['SU_leader', 'Andropov', 'Brezhnev', 'Chernenko',
       'Gorbachev', 'Khrushchev', 'Stalin']] = wim[
    ['SU_leader', 'Andropov', 'Brezhnev', 'Chernenko',
       'Gorbachev', 'Khrushchev', 'Stalin']].ffill()

# Populate A variable that tracks the current SU leaders years in office
wim['SUL_start_year'] = wim['SUL_start_year'].ffill()
wim['SUL_YIO'] = wim['year'] - wim['SUL_start_year'] 

### Add GDP and CINC scores, both as target and dummy variables

In [5]:
col_powers = {'United Kingdom': 200, 'Netherlands' : 210, 'Belgium': 211, 'France': 220, 'Spain': 230, 'Portugal': 235}

# Get CINC scores for target nations
cinc = pd.read_csv('./sources/NMC.csv')
mask = cinc['year'] < 1992
cinc = cinc[mask]
mask = cinc['year'] > 1945
cinc = cinc[mask]
cinc = cinc[['ccode', 'year', 'milex', 'cinc']]

def gdp_globals(code, name):
    temp = wim[wim['cowcode'] == code]
    temp = temp[['gdppc', 'gdppcl', 'year']]
    temp.columns = [name + '_' + str(x) for x in temp.columns]
    temp.reset_index(inplace=True)
    return temp

def cinc_globals(code, name):
    temp = cinc[cinc['ccode'] == code]
    temp = temp[['milex', 'cinc', 'year']]
    temp.columns = [name + '_' + str(x) for x in temp.columns]
    temp.reset_index(inplace=True)
    return temp    
    
# Add country specific GDP variables
us_gdp = gdp_globals(2, 'US')
ru_gdp = gdp_globals(365, 'SU')
uk_gdp = gdp_globals(200, 'UK')
nl_gdp = gdp_globals(210, 'NL')
be_gdp = gdp_globals(211, 'BE')
fr_gdp = gdp_globals(220, 'FR')
sp_gdp = gdp_globals(230, 'SP')
pr_gdp = gdp_globals(235, 'PR')
cuba_gdp = gdp_globals(40, 'CUBA')
ch_gdp = gdp_globals(710, 'CH')
yg_gdp = gdp_globals(345, 'YG')

gdp_merged = pd.concat([
    us_gdp, ru_gdp, uk_gdp, nl_gdp, be_gdp,
    fr_gdp, sp_gdp, pr_gdp, cuba_gdp, ch_gdp, yg_gdp
], axis=1)

us_cinc = cinc_globals(2, 'US')
ru_cinc = cinc_globals(365, 'SU')
uk_cinc = cinc_globals(200, 'UK')
nl_cinc = cinc_globals(210, 'NL')
be_cinc = cinc_globals(211, 'BE')
fr_cinc = cinc_globals(220, 'FR')
sp_cinc = cinc_globals(230, 'SP')
pr_cinc = cinc_globals(235, 'PR')
cuba_cinc = cinc_globals(40, 'CUBA')
ch_cinc = cinc_globals(710, 'CH')
yg_cinc = cinc_globals(345, 'YG')

cinc_merged = pd.concat([
    us_cinc, ru_cinc, uk_cinc, nl_cinc, be_cinc,
    fr_cinc, sp_cinc, pr_cinc, cuba_cinc, ch_cinc, yg_cinc
], axis=1)
gpd_cinc_merged = pd.concat([gdp_merged, cinc_merged], axis=1)
gpd_cinc_merged.drop('index', axis=1, inplace=True)
gpd_cinc_merged.drop([
    'SU_year', 'UK_year', 'BE_year', 'FR_year', 'SP_year',
    'PR_year', 'NL_year', 'CUBA_year', 'CH_year', 'YG_year'
], axis=1, inplace=True)

# Remove duplicate columns. 
c = pd.Index(['US_gdppc', 'US_gdppcl', 'US_year', 'SU_gdppc', 'SU_gdppcl', 'UK_gdppc',
       'UK_gdppcl', 'NL_gdppc', 'NL_gdppcl', 'BE_gdppc', 'BE_gdppcl',
       'FR_gdppc', 'FR_gdppcl', 'FR_year', 'SP_gdppc', 'SP_gdppcl',
       'PR_gdppc', 'PR_gdppcl', 'US_milex', 'US_cinc', 'US_year', 'U_milex',
       'SU_cinc', 'UK_milex', 'UK_cinc', 'NL_milex', 'NL_cinc', 'BE_milex',
       'BE_cinc', 'FR_milex', 'FR_cinc', 'SP_milex', 'SP_cinc', 'PR_milex',
       'PR_cinc'])

gpd_cinc_merged = gpd_cinc_merged.loc[:, ~gpd_cinc_merged.columns.duplicated()]

#Merge new dataframe into Wimmers dataset
wim = wim.merge(gpd_cinc_merged, left_on='year', right_on='US_year', how='left')
# Add cinc and milex variables for indivual nations as well. 
wim = wim.merge(cinc, left_on=['year', 'cowcode'], right_on=['year', 'ccode'], how='left')

# Merge UCDP external support into the BaseDataset

In [6]:
exsup = pd.read_csv('./output/EXSUP_PREPARED.csv')
# Reset index so 'locationid1' becomes available again.
exsup.reset_index(inplace=True)
merged = wim.merge(exsup, left_on=['cowcode', 'year'],
                   right_on=['locationid1', 'ywp_year'], how='left')
# External exists is a value that we dont want Null values for, ffill them. 
# merged['external_exists'].fillna(value=0, inplace=True)

# Merge the IMI intervention dataset into the BaseDateset

In [7]:
imi = pd.read_csv('./output/IMI_prepared.csv')
base_df = merged.merge(imi, left_on=['cowcode', 'year'], right_on=['target', 'date'], how='left')
base_df[imi.columns] = base_df[imi.columns].fillna(0)

# Add Yugoslavia
base_df['Yugoslavia'] = 0

# FIllna first, function doesnt work on NaN values
base_df['United States'].fillna(value=0, inplace=True)
base_df['Soviet Union'].fillna(value=0, inplace=True)
base_df['Cuba'].fillna(value=0, inplace=True)
base_df['China'].fillna(value=0, inplace=True)
base_df['Yugoslavia'].fillna(value=0, inplace=True)
base_df['Soviet Union'].fillna(value=0, inplace=True)
base_df['United Kingdom'].fillna(value=0, inplace=True)
base_df['France'].fillna(value=0, inplace=True)
base_df['Netherlands'].fillna(value=0, inplace=True)
base_df['Belgium'].fillna(value=0, inplace=True)
base_df['Spain'].fillna(value=0, inplace=True)
base_df['Portugal'].fillna(value=0, inplace=True)

# Edit exising US/ SU dummies
def US_checker(x):
    if x['US_imi'] == 1:
        return 1
    else:
        return x['US_troops']

def SU_checker(x):
    if x['SU_imi'] == 1:
        return 1
    else:
        return x['SU_troops']
    
def FR_checker(x):
    if x['FR_imi'] == 1:
        return 1
    else:
        return x['FR_troops']

def UK_checker(x):
    if x['UK_imi'] == 1:
        return 1
    else:
        return x['UK_troops']

def BE_checker(x):
    if x['BE_imi'] == 1:
        return 1
    else:
        return x['BE_troops']

def CUBA_checker(x):
    if x['CUBA_imi'] == 1:
        return 1
    else:
        return x['CUBA_troops']
    
def CH_checker(x):
    if x['CH_imi'] == 1:
        return 1
    else:
        return x['CH_troops']

def YG_checker(x):
    if x['YG_imi'] == 1:
        return 1
    else:
        #YG not included in other dataset, so auto return 0
        return 0

base_df['United States'] = base_df.apply(US_checker, axis=1)
base_df['Soviet Union'] = base_df.apply(SU_checker, axis=1)
base_df['United Kingdom'] = base_df.apply(UK_checker, axis=1)
base_df['France'] = base_df.apply(FR_checker, axis=1)
# base_df['Belgium'] = base_df.apply(BE_checker, axis=1)
base_df['China'] = base_df.apply(CH_checker, axis=1)
base_df['Cuba'] = base_df.apply(CUBA_checker, axis=1)
base_df['Yugoslavia'] = base_df.apply(YG_checker, axis=1)

# # Recode dummies (Recheck later)
col_list = ['United Kingdom', 'France', 'Belgium', 'Netherlands', 'Spain', 'Portugal']
base_df['COL_IMI'] = (base_df[col_list].sum(axis=1) > 0) * 1 

com_list = ['CUBA_imi', 'CH_imi', 'YG_imi']
base_df['COM_IMI'] = (base_df[com_list].sum(axis=1) > 0) * 1 

# Remove duplicate target column
base_df.drop(['Unnamed: 0'], axis=1, inplace=True)

## Add 'number of civil wars' column to dataset

In [8]:
# Add 'number of wars' per year to dataset
N_Wars =  base_df.groupby('year').agg({
    'ocivil': 'sum',
    'pocivil': 'sum',
    'ointrap': 'sum',
    'pointrap': 'sum', 
    'ononind': 'sum',
    'pononind': 'sum',
    'onatind': 'sum',
    'ponatind': 'sum',    
}).reset_index()

N_Wars['n_civilwars'] = N_Wars[[
    'pointrap', 'onatind', 'ocivil', 'ponatind',
    'pocivil', 'pononind', 'ointrap', 'ononind'
]].sum(axis=1)

# Select only the columns that we are going to use
# to prevent duplicates
N_Wars = N_Wars[['year', 'n_civilwars']]
base_df = base_df.merge(N_Wars, left_on='year', right_on='year', how='left')

## Code dummy variables for ANY FORM OF intervention in ALL wars

In [9]:
# Add dummies for the various types of Civil Wars
dummies = pd.get_dummies(wim['wartype'])
base_df = pd.concat([base_df, dummies], axis=1)

# Add Dummy variable for 'civil wars'
base_df['DUMMY_CIVIL'] = (base_df[['CIVIL', 'NATIND', 'NONIND']].sum(axis=1) > 0) * 1 

In [10]:
base_df['US_ci'] = 0
base_df['SU_ci'] = 0
base_df['COL_ci'] = 0
base_df['FR_ci'] = 0
base_df['UK_ci'] = 0

warlist = [
    'ocivil', 'pocivil', 'ointrap', 'pointrap',
    'ononind', 'pononind', 'onatind', 'ponatind'
]

def US_ci(x):
    status = 0
    if x['United States'] == 1:
        for war in warlist:
            if x[war] == 1:
                status = 1
            else:
                pass
        return status
    else:
        return 0

    
def SU_ci(x):
    status = 0
    if x['Soviet Union'] == 1:
        for war in warlist:
            if x[war] == 1:
                status = 1
            else:
                pass
        return status
    else:
        return 0

def COL_ci(x):
    status = 0
    if x['COL_IMI'] == 1:
        for war in warlist:
            if x[war] == 1:
                status = 1
            else:
                pass
        return status
    else:
        return 0
    
    
def FR_ci(x):
    status = 0
    if x['France'] == 1:
        for war in warlist:
            if x[war] == 1:
                status = 1
            else:
                pass
        return status
    else:
        return 0
    
    
def UK_ci(x):
    status = 0
    if x['United Kingdom'] == 1:
        for war in warlist:
            if x[war] == 1:
                status = 1
            else:
                pass
        return status
    else:
        return 0

In [11]:
base_df['US_ci'] = base_df.apply(US_ci, axis=1)
base_df['SU_ci'] = base_df.apply(SU_ci, axis=1)
base_df['COL_ci'] = base_df.apply(COL_ci, axis=1)
base_df['FR_ci'] = base_df.apply(FR_ci, axis=1)
base_df['UK_ci'] = base_df.apply(UK_ci, axis=1)

### Add 'Country X - current interventions variables

In [12]:
summed_interventions = base_df.groupby('year').agg({
    'US_ci': 'sum',
    'SU_ci': 'sum',
    'FR_ci': 'sum',
    'UK_ci': 'sum',
    'COL_ci': 'sum', 
})

summed_interventions.reset_index(inplace=True)

## Add a generic 'colony' dummy

In [13]:
def colchecker(x):
    if x['implag'] > 0:
        return 1
    else:
        return 0

base_df['was_colony'] = base_df.apply(colchecker, axis=1)

In [14]:
# Rename columns to prevent duplicates 
summed_interventions = summed_interventions.add_prefix('summed_')
# Pick only the columns that we need
summed_interventions

summed_interventions = summed_interventions[
    ['summed_year', 'summed_COL_ci', 'summed_US_ci',
     'summed_UK_ci', 'summed_SU_ci', 'summed_FR_ci']
]

base_df = base_df.merge(summed_interventions, left_on='year', right_on='summed_year')
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6900 entries, 0 to 6899
Columns: 293 entries, yearc to summed_FR_ci
dtypes: float64(222), int64(59), object(7), uint8(5)
memory usage: 15.2+ MB


## Add 'War Duration' variable

In [15]:
base_df['war_duration'] = base_df['yrend'] - base_df['yrbeg'] + 1

## Add grouped cinc_scores

In [16]:
# Cinc score for colonial powers
base_df['COL_cinc'] = base_df['FR_cinc'] + base_df['UK_cinc'] + base_df['PR_cinc']
# Cinc score for thirdworld communist
base_df['COM_cinc'] = base_df['CUBA_cinc'] + base_df['CH_cinc'] + base_df['YG_cinc']

## Add grouped milex_scores

In [17]:
# Milex score for colonial powers
base_df['COL_milex'] = base_df['FR_milex'] + base_df['UK_milex'] + base_df['PR_milex']
# Milex score for thirdworld communist
base_df['COM_milex'] = base_df['CUBA_milex'] + base_df['CH_milex'] + base_df['YG_milex']

## Add 'Any' intervention 

In [18]:
base_df['US_any'] = base_df['United States'] + base_df['US_none_mil']
base_df['SU_any'] = base_df['Soviet Union'] + base_df['SU_none_mil']

### Save Dataset to file

In [19]:
# Save to CSV
base_df.to_csv('./output/masterfile.csv')

### The full list of variables that are present in the Dataset

In [20]:
for x in base_df.columns:
    print(x)

yearc
year
cowcode
country
onset
war
warname
warno
wartype
yrbeg
yrend
anarc
anarcl
anoc
anocl
area2001
asia
autoc
autocl
democ
democl
eeurop
ethfrac
gdp
gdppc
gdppcl
implag
imppower
instab
instabl
lamerica
lmtnest
lnpop
lnpopl
milperc
milpercl
nafrme
nbcivil
nbconq
nbinter
nbnatind
nbnonind
nsflag
nsfyear
ocivil
oconq
oil
oilpc
oilpcl
ointer
ointrap
onatind
ononind
pdemnb
pocivil
poconq
pointer
pointrap
poldisc
poldiscl
ponatind
pononind
ponset
pop
relfrac
ssafrica
western
start
president
party
Dwight D. Eisenhower
George H. W. Bush
Gerald Ford
Harry S. Truman
Jimmy Carter
John F. Kennedy
Lyndon B. Johnson
Richard Nixon
Ronald Reagan
Democratic
Republican
USP_YIO
SU_leader
SUL_start_year
Andropov
Brezhnev
Chernenko
Gorbachev
Khrushchev
Stalin
SUL_YIO
US_gdppc
US_gdppcl
US_year
SU_gdppc
SU_gdppcl
UK_gdppc
UK_gdppcl
NL_gdppc
NL_gdppcl
BE_gdppc
BE_gdppcl
FR_gdppc
FR_gdppcl
SP_gdppc
SP_gdppcl
PR_gdppc
PR_gdppcl
CUBA_gdppc
CUBA_gdppcl
CH_gdppc
CH_gdppcl
YG_gdppc
YG_gdppcl
US_milex
US_cinc


In [21]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6900 entries, 0 to 6899
Columns: 300 entries, yearc to SU_any
dtypes: float64(227), int64(61), object(7), uint8(5)
memory usage: 15.6+ MB
